In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers bitsandbytes accelerate datasets peft unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/8

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# ✅ Step 1: Load Dataset
data_path = "2501.12948v1_qa_pairs.json"  # Update with your dataset path
dataset = load_dataset("json", data_files=data_path)

# ✅ Step 2: Split Dataset into Train & Test
dataset = dataset["train"].train_test_split(test_size=0.1)

# ✅ Step 3: Load Tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Step 4: Define Preprocessing Function
def preprocess_data(examples):
    """Formats instruction-based dataset into a dictionary for training."""
    prompts = []
    outputs = []

    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput: "

        # Tokenize input (Prompt)
        tokenized_prompt = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)

        # Tokenize output (Expected response)
        tokenized_output = tokenizer(output, padding="max_length", truncation=True, max_length=512)

        # Append to lists
        prompts.append(tokenized_prompt["input_ids"])
        outputs.append(tokenized_output["input_ids"])

    # ✅ Return a dictionary instead of a list (Fixes TypeError)
    return {
        "input_ids": prompts,
        "labels": outputs
    }

# ✅ Step 5: Apply Tokenization to the Dataset
train_dataset = dataset["train"].map(
    preprocess_data,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)

test_dataset = dataset["test"].map(
    preprocess_data,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)

# ✅ Step 6: Print Example to Verify
print(train_dataset[0])  # Check structure



Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Map:   0%|          | 0/378 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

{'input_ids': [16664, 25, 3555, 374, 18090, 2668, 52, 5267, 2505, 25, 2157, 68674, 10951, 84260, 1073, 4128, 8660, 13, 18090, 2668, 52, 25, 2157, 300, 324, 12, 6749, 10951, 84260, 1073, 4128, 8660, 304, 8453, 624, 5097, 25, 220, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151

In [ ]:
import torch
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

import os
os.environ["WANDB_DISABLED"] = "true"


# ✅ Load Qwen Model with 4-bit Quantization (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# ✅ Prepare Model for Training
model = prepare_model_for_kbit_training(model)

# ✅ Attach LoRA Adapters
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,  # Small batch size due to limited VRAM
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Simulate larger batch size
    logging_dir="./logs",
    fp16=True,
    num_train_epochs=3,
    save_total_limit=1,
    optim="adamw_bnb_8bit",
    remove_unused_columns=False,  # Prevent Trainer from removing dataset columns
)

# ✅ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# ✅ Free GPU Memory Before Training
torch.cuda.empty_cache()

# ✅ Start Training
trainer.train()

# ✅ Save LoRA Adapters
model.save_pretrained("/content/drive/MyDrive/ML_Stuff/Qwen/fine_tuned_qwen_lora")
tokenizer.save_pretrained("/content/drive/MyDrive/ML_Stuff/Qwen/fine_tuned_qwen_lora")

print("🎉 Fine-tuning completed with QLoRA! LoRA adapters saved.")


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,2.497548
2,No log,1.540003


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,2.497548
2,No log,1.267539


🎉 Fine-tuning completed with QLoRA! LoRA adapters saved.


In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

# Load base model
model_name = "Qwen/Qwen2.5-3B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda:0")

# Load fine-tuned LoRA adapters
fine_tuned_lora_path = "/content/drive/MyDrive/ML_Stuff/Qwen/fine_tuned_qwen_lora"
model = PeftModel.from_pretrained(base_model, fine_tuned_lora_path)

# Merge LoRA weights into the base model
model = model.merge_and_unload()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM
from peft import PeftModel

# ✅ Check RAM & Disk Space Before Merging
os.system("free -h")  # Show available RAM
os.system("df -h")    # Show disk space usage

# ✅ Load Base Model with Memory Optimization
model_name = "Qwen/Qwen2.5-3B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Reduce memory usage
    device_map="cpu"  # Load on CPU to prevent GPU crashes
)

# ✅ Free GPU Memory Before Merging (if using Colab/Kaggle)
torch.cuda.empty_cache()

# ✅ Load Fine-Tuned LoRA Adapters
fine_tuned_lora_path = "/content/drive/MyDrive/ML_Stuff/Qwen/fine_tuned_qwen_lora"
model = PeftModel.from_pretrained(base_model, fine_tuned_lora_path).to("cpu")

# ✅ Merge LoRA Weights into Base Model
print("Merging LoRA weights into base model... This may take some time.")
model = model.merge_and_unload()

# ✅ Save the Fully Merged Model
merged_model_path = "/content/drive/MyDrive/ML_Stuff/Qwen/qwen2.5_3b_merged"
model.save_pretrained(merged_model_path)

print(f"🎉 LoRA weights merged successfully! Model saved at {merged_model_path}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Merging LoRA weights into base model... This may take some time.
🎉 LoRA weights merged successfully! Model saved at /content/drive/MyDrive/ML_Stuff/Qwen/qwen2.5_3b_merged


In [ ]:
!pip install llama-cpp-python transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4552804 sha256=3d64a10324b6a302d32046b6902e9d39c5873622d1c738228437e9de1f4ba540
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [ ]:
# Install required packages
!apt-get install -y cmake build-essential
!pip install llama-cpp-python
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4601124 sha256=7e196077b48d6860f27378024bd04766948c0d5185882d3ef9d623e10cb11aac
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating obj

In [ ]:
!mkdir build
%cd build
!cmake ..
!cmake --build . --config Release
%cd /content/

/content/llama.cpp/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP:

In [ ]:
!pip install ctranslate2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 23.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import subprocess

model_path = "/content/drive/MyDrive/ML_Stuff/Qwen/qwen2.5_3b_merged"  # Replace with your model path
converted_model_path = model_path + "_converted"
gguf_model_path = model_path + "_gguf.gguf"

# Load model in a memory-efficient way (handles multiple safetensors automatically)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",  # Use automatic precision to reduce memory usage
    low_cpu_mem_usage=True,  # Optimize memory during loading
    device_map="auto",  # Load model on available GPU/CPU
    use_safetensors=True  # Ensure safetensors are correctly loaded
)

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Save the model in ctranslate2 format
model.save_pretrained(
    converted_model_path + "_ctranslate2",
    from_pt=True,
    max_shard_size="500MB"  # Reduce shard size to prevent memory overflow
)
tokenizer.save_pretrained(converted_model_path + "_ctranslate2")

# Convert to GGUF using llama.cpp
subprocess.run([
    "/content/llama.cpp/convert_hf_to_gguf.py", model_path, "--outfile", gguf_model_path
])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CompletedProcess(args=['/content/llama.cpp/convert_hf_to_gguf.py', '/content/drive/MyDrive/ML_Stuff/Qwen/qwen2.5_3b_merged', '--outfile', '/content/drive/MyDrive/ML_Stuff/Qwen/qwen2.5_3b_merged_gguf'], returncode=0)